## Preprocesamiento

In [9]:
import pandas as pd
from PIL import Image
import numpy as np

In [2]:
dataset = pd.read_csv('data/data.csv')

In [3]:
numeric_features = ['extent']
categorical_features = ['growth_stage', 'damage', 'season']

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

transformers_encoder_onehot = [
    ('encoder', OneHotEncoder(), categorical_features)
]
column_transformer = ColumnTransformer(transformers_encoder_onehot, remainder='passthrough')

In [5]:
dataset_encoded = column_transformer.fit_transform(dataset)
dataset = pd.DataFrame(dataset_encoded, columns=column_transformer.get_feature_names_out())

In [6]:
new_column_names = [col.replace('encoder__', '').replace('remainder__', '') for col in column_transformer.get_feature_names_out()]
dataset.columns = new_column_names

In [7]:
dataset = dataset.drop(columns=['ID'])

In [8]:
dataset.head()

,growth_stage_F,growth_stage_M,growth_stage_S,growth_stage_V,damage_DR,damage_DS,damage_FD,damage_G,damage_ND,damage_PS,damage_WD,damage_WN,season_LR2020,season_LR2021,season_SR2020,season_SR2021,filename,extent
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,L427F01330C01S03961Rp02052.jpg,0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,L1083F00930C39S12674Ip.jpg,0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24_initial_1_1463_1463.JPG,0
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,L341F00167C01S00324Rp14178.jpg,60
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,L1084F02394C39S13931Ip.jpg,0


In [15]:
def resizeImg(path, size):
    img = Image.open(path)

    width, height = img.size
    aspectRatio = width / height

    if aspectRatio < 1:
        nHeight = size
        nWidth = int(size * aspectRatio)
    else:
        nWidth = size
        nHeight = int(size / aspectRatio)

    resizedImg = img.resize((nWidth, nHeight), Image.LANCZOS)

    newImg = Image.new("RGB", (size, size), (255, 255, 255))

    xOffset = (size - nWidth) // 2
    yOffset = (size - nHeight) // 2
    newImg.paste(resizedImg, (xOffset, yOffset))

    return newImg

In [ ]:
import os
from tqdm import tqdm

# Ruta donde se encuentran las imágenes
image_path = "data/train/"
image_size = 128
output_file = "resized_images.npy"

def load_and_resize_images(df, path, size):
    images = []
    for filename in tqdm(df['filename']):
        img = resizeImg(os.path.join(path, filename), size)
        img_array = np.array(img) / 255.0
        images.append(img_array)
    return np.array(images)

# Verificar si ya existe el archivo .npy
if not os.path.exists(output_file):
    images = load_and_resize_images(dataset, image_path, image_size)
    np.save(output_file, images)  # Guardar el archivo .npy
else:
    images = np.load(output_file)  # Cargar el archivo .npy


## CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Cargar el modelo preentrenado VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
# Extraer las características de la última capa convolucional
model = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)

# Extraer características
features = model.predict(images)
# Aplanar las características para usarlas en Random Forest
features_flat = features.reshape(len(features), -1)

In [ ]:
# Convertir las características a un DataFrame y unir con el dataset original
features_df = pd.DataFrame(features_flat)
df_features = pd.concat([df.reset_index(drop=True), features_df], axis=1)
df_features.drop(columns=["filename"], inplace=True)  # Retirar la columna 'filename' ya que ahora tenemos las features

## Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Dividir datos
X = df_features.drop(columns=["extent"])
y = df_features["extent"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model_rf.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")